# Preprocessing data

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.cores", "4") \
    .appName("Product Recommendation") \
    .getOrCreate()
df = spark.read.csv("/home/m1nhd3n/Works/DataEngineer/product_recommendations/data/raw/data.csv", header=True, inferSchema=True)


25/02/20 14:32:42 WARN Utils: Your hostname, m1nhd3n resolves to a loopback address: 127.0.1.1; using 192.168.1.158 instead (on interface wlp0s20f3)
25/02/20 14:32:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/20 14:32:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## 1. Overview
### 1.1. Schema

In [2]:
df.printSchema()

root
 |-- fecha_dato: date (nullable = true)
 |-- ncodpers: double (nullable = true)
 |-- ind_empleado: string (nullable = true)
 |-- pais_residencia: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- age: string (nullable = true)
 |-- fecha_alta: date (nullable = true)
 |-- ind_nuevo: string (nullable = true)
 |-- antiguedad: string (nullable = true)
 |-- indrel: string (nullable = true)
 |-- ult_fec_cli_1t: date (nullable = true)
 |-- indrel_1mes: string (nullable = true)
 |-- tiprel_1mes: string (nullable = true)
 |-- indresi: string (nullable = true)
 |-- indext: string (nullable = true)
 |-- conyuemp: string (nullable = true)
 |-- canal_entrada: string (nullable = true)
 |-- indfall: string (nullable = true)
 |-- tipodom: string (nullable = true)
 |-- cod_prov: string (nullable = true)
 |-- nomprov: string (nullable = true)
 |-- ind_actividad_cliente: string (nullable = true)
 |-- renta: double (nullable = true)
 |-- segmento: string (nullable = true)
 |-- ind_ahor

### 1.2. EDA

In [3]:
df.count()

13647309

Dataset contains monthly record of customers' behaviour and possessed products, from 28-02-2015 to 28-05-2016

In [4]:
df.select("fecha_dato").distinct().show()

+----------+
|fecha_dato|
+----------+
|2015-02-28|
|2015-01-28|
|2015-03-28|
|2015-04-28|
|2015-06-28|
|2015-05-28|
|2015-07-28|
|2015-08-28|
|2015-09-28|
|2015-10-28|
|2015-11-28|
|2015-12-28|
|2016-01-28|
|2016-02-28|
|2016-03-28|
|2016-04-28|
|2016-05-28|
+----------+



In [41]:
print(f"Number of customers: {df.select('ncodpers').distinct().count()}")

Number of customers: 956645


Age column contains null values.

In [42]:
df.select("sexo").distinct().show()

+----+
|sexo|
+----+
|   V|
|   H|
|NULL|
+----+



Some columns contains NA values. We must transform these NA values into Null values.

In [ ]:
df.select("ind_nuevo").distinct().show()

+---------+
|ind_nuevo|
+---------+
|       NA|
|        1|
|        0|
+---------+



In [46]:
df.select("indrel").distinct().show()

+------+
|indrel|
+------+
|    NA|
|    99|
|     1|
+------+



In [47]:
df.select("indrel_1mes").distinct().show()

+-----------+
|indrel_1mes|
+-----------+
|        1.0|
|          3|
|          1|
|        3.0|
|          2|
|        4.0|
|        2.0|
|          P|
|          4|
|       NULL|
+-----------+



In [48]:
df.select("tiprel_1mes").distinct().show()

+-----------+
|tiprel_1mes|
+-----------+
|          A|
|          I|
|          P|
|          R|
|          N|
|       NULL|
+-----------+



In [49]:
df.select("indresi").distinct().show()

+-------+
|indresi|
+-------+
|      N|
|      S|
|   NULL|
+-------+



In [50]:
df.select("indext").distinct().show()

+------+
|indext|
+------+
|     N|
|     S|
|  NULL|
+------+



In [52]:
df.select("conyuemp").distinct().show()

+--------+
|conyuemp|
+--------+
|       N|
|       S|
|    NULL|
+--------+



In [53]:
df.select("canal_entrada").distinct().count()

163

In [54]:
df.select("ind_actividad_cliente").distinct().show()

+---------------------+
|ind_actividad_cliente|
+---------------------+
|                   NA|
|                    1|
|                    0|
+---------------------+



In [55]:
df.select("segmento").distinct().show()

+------------------+
|          segmento|
+------------------+
|          01 - TOP|
| 02 - PARTICULARES|
|03 - UNIVERSITARIO|
|              NULL|
+------------------+



### 1.3. Types Consistency

For numerical columns, we can just cast them into numbers and then those NA values will automatically be null. For columns that is already contain null values instead of NA, we don't need to do anything at this step.

In [56]:
df = df.replace("P", "5", subset=["indrel_1mes"])

df = df.withColumns({
    "age": df.age.cast("int"),
    "ind_nuevo": df.ind_nuevo.cast("int"),
    "indrel": df.indrel.cast("int"),
    "ind_actividad_cliente": df.ind_actividad_cliente.cast("int"),
    "antiguedad": df.antiguedad.cast("int"),
    "indrel_1mes": df.indrel_1mes.cast("int")
})

df.select("ind_nuevo").distinct().show()
df.select("indrel").distinct().show()
df.select("ind_actividad_cliente").distinct().show()
df.select("indrel_1mes").distinct().show()

+---------+
|ind_nuevo|
+---------+
|        1|
|        0|
|     NULL|
+---------+



+------+
|indrel|
+------+
|     1|
|    99|
|  NULL|
+------+



+---------------------+
|ind_actividad_cliente|
+---------------------+
|                    1|
|                    0|
|                 NULL|
+---------------------+



+-----------+
|indrel_1mes|
+-----------+
|          1|
|          3|
|          2|
|          5|
|          4|
|       NULL|
+-----------+



In [57]:
df = df.withColumns({
    "ind_nomina_ult1": df.ind_nomina_ult1.cast("int"),
    "ind_nom_pens_ult1": df.ind_nom_pens_ult1.cast("int")
})
df.select("ind_nomina_ult1").distinct().show()
df.select("ind_nom_pens_ult1").distinct().show()

+---------------+
|ind_nomina_ult1|
+---------------+
|              1|
|              0|
|           NULL|
+---------------+



+-----------------+
|ind_nom_pens_ult1|
+-----------------+
|                1|
|                0|
|             NULL|
+-----------------+



Transform S/N into 0, 1

In [59]:
sn_cols = ['indresi', 'indext', 'conyuemp', 'indfall']

df = df.replace("N", "0", subset=sn_cols)
df = df.replace("S", "1", subset=sn_cols)

df.select("indresi").distinct().show()
df.select("indext").distinct().show()
df.select("conyuemp").distinct().show()
df.select("indfall").distinct().show()

+-------+
|indresi|
+-------+
|      0|
|      1|
|   NULL|
+-------+



+------+
|indext|
+------+
|     0|
|     1|
|  NULL|
+------+



+--------+
|conyuemp|
+--------+
|       0|
|       1|
|    NULL|
+--------+



+-------+
|indfall|
+-------+
|      0|
|      1|
|   NULL|
+-------+



In [60]:
df = df.withColumns({
    "indresi": df.indresi.cast("int"),
    "indext": df.indext.cast("int"),
    "conyuemp": df.conyuemp.cast("int"),
    "indfall": df.indfall.cast("int")
})

In [64]:
df = df.withColumn("ncodpers", df.ncodpers.cast("int"))

In [65]:
df.show()

+----------+--------+------------+---------------+----+---+----------+---------+----------+------+--------------+-----------+-----------+-------+------+--------+-------------+-------+-------+--------+-----------+---------------------+------------------+------------------+-----------------+-----------------+----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+---------------+-----------------+---------------+
|fecha_dato|ncodpers|ind_empleado|pais_residencia|sexo|age|fecha_alta|ind_nuevo|antiguedad|indrel|ult_fec_cli_1t|indrel_1mes|tiprel_1mes|indresi|indext|conyuemp|canal_entrada|indfall|tipodom|cod_prov|    nomprov|ind_actividad_cliente|             renta|          segmento|ind_ahor_fin_ult1|ind_aval_fin_

In [82]:
df = df.replace("NA", None, subset=["tipodom", "cod_prov"])

### 1.3. Null Counts

In [83]:
from pyspark.sql.functions import col, sum

df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

+----------+--------+------------+---------------+-----+-----+----------+---------+----------+------+--------------+-----------+-----------+-------+------+-------------+-------+-------+--------+-------+---------------------+-------+--------+-----------------+-----------------+----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+---------------+-----------------+---------------+
|fecha_dato|ncodpers|ind_empleado|pais_residencia| sexo|  age|fecha_alta|ind_nuevo|antiguedad|indrel|ult_fec_cli_1t|indrel_1mes|tiprel_1mes|indresi|indext|canal_entrada|indfall|tipodom|cod_prov|nomprov|ind_actividad_cliente|  renta|segmento|ind_ahor_fin_ult1|ind_aval_fin_ult1|ind_cco_fin_ult1|ind_cder_fin_ult1|ind_cno_fin_ult1|ind_c

It seems like many rows are broken, where all columns are null

In [84]:
df.select("*").where(df.age.isNull()).show()

+----------+--------+------------+---------------+----+----+----------+---------+----------+------+--------------+-----------+-----------+-------+------+-------------+-------+-------+--------+-------+---------------------+-----+--------+-----------------+-----------------+----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+---------------+-----------------+---------------+
|fecha_dato|ncodpers|ind_empleado|pais_residencia|sexo| age|fecha_alta|ind_nuevo|antiguedad|indrel|ult_fec_cli_1t|indrel_1mes|tiprel_1mes|indresi|indext|canal_entrada|indfall|tipodom|cod_prov|nomprov|ind_actividad_cliente|renta|segmento|ind_ahor_fin_ult1|ind_aval_fin_ult1|ind_cco_fin_ult1|ind_cder_fin_ult1|ind_cno_fin_ult1|ind_ctju_fin_

In [85]:
from pyspark.sql import functions as F

possible_all_nulls_col = [
    'ind_empleado',
    'pais_residencia',
    'sexo',
    'age',
    'fecha_alta',
    'ind_nuevo',
    'antiguedad',
    'indrel',
    'ult_fec_cli_1t',
    'indrel_1mes',
    'tiprel_1mes',
    'indresi',
    'indext',
    'canal_entrada',
    'indfall',
    'tipodom',
    'cod_prov',
    'nomprov',
    'ind_actividad_cliente',
    'renta',
    'segmento'
]
test_df = df
for c in possible_all_nulls_col:
    test_df = test_df.select("*").where(F.col(c).isNull())

test_df.count()

27734

In [88]:
null_age_df = df.select("*").where(df.age.isNull())

In [89]:
test_df = null_age_df
for c in possible_all_nulls_col:
    test_df = test_df.select("*").where(F.col(c).isNull())

test_df.count()

27734

It seems like all the rows with null age are broken. We will simply remove these rows.

In [90]:
df = df.select("*").where(df.age.isNotNull())

In [91]:
df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

+----------+--------+------------+---------------+----+---+----------+---------+----------+------+--------------+-----------+-----------+-------+------+-------------+-------+-------+--------+-------+---------------------+-------+--------+-----------------+-----------------+----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+---------------+-----------------+---------------+
|fecha_dato|ncodpers|ind_empleado|pais_residencia|sexo|age|fecha_alta|ind_nuevo|antiguedad|indrel|ult_fec_cli_1t|indrel_1mes|tiprel_1mes|indresi|indext|canal_entrada|indfall|tipodom|cod_prov|nomprov|ind_actividad_cliente|  renta|segmento|ind_ahor_fin_ult1|ind_aval_fin_ult1|ind_cco_fin_ult1|ind_cder_fin_ult1|ind_cno_fin_ult1|ind_ctju_fi

Notice `conyuemp` and `ult_fec_cli_1t` has so many null values. `conyuemp` field seem broken so we should drop this column

In [68]:
df.select("*").groupBy(df.conyuemp).count().show()

+--------+--------+
|conyuemp|   count|
+--------+--------+
|    NULL|13645501|
|       1|      17|
|       0|    1791|
+--------+--------+



In [69]:
df = df.drop("conyuemp")

Based on the description of the dataset, it seems like `indrel` and `ult_fec_cli_1t` has some kind of relationship:
- If `ult_fec_cli_1t` is Null then `indrel` is 1.
- If `ult_fec_cli_1t` is not Null then `indrel` is 99.

In [92]:
df.select("indrel", "ult_fec_cli_1t").where(df.ult_fec_cli_1t.isNull()).where(df.indrel == 1).count()

13594782

In [93]:
df.select("indrel", "ult_fec_cli_1t").where(df.ult_fec_cli_1t.isNotNull()).where(df.indrel == 99).count()

24793

In [94]:
df.count() - (13594782 + 24793)

0

The information in column `ult_fec_cli_1t` is being represent by `indrel` so we can drop this column also.

In [95]:
df = df.drop("ult_fec_cli_1t")

In [96]:
df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

+----------+--------+------------+---------------+----+---+----------+---------+----------+------+-----------+-----------+-------+------+-------------+-------+-------+--------+-------+---------------------+-------+--------+-----------------+-----------------+----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+---------------+-----------------+---------------+
|fecha_dato|ncodpers|ind_empleado|pais_residencia|sexo|age|fecha_alta|ind_nuevo|antiguedad|indrel|indrel_1mes|tiprel_1mes|indresi|indext|canal_entrada|indfall|tipodom|cod_prov|nomprov|ind_actividad_cliente|  renta|segmento|ind_ahor_fin_ult1|ind_aval_fin_ult1|ind_cco_fin_ult1|ind_cder_fin_ult1|ind_cno_fin_ult1|ind_ctju_fin_ult1|ind_ctma_fin_ult1|ind_c

Drop the rows where sex is Null since only 70 of them is null

In [97]:
df = df.select("*").where(df.sexo.isNotNull())

Let's see if `indrel_1mes` has any relationship with `tiprel_1mes`. It seems like if one of these column is Null then the other is also Null. We should also drop these columns since these null rows only take up less than 1% of the dataset.

In [99]:
test_df = df.where(df.indrel_1mes.isNull())
test_df = test_df.where(test_df.tiprel_1mes.isNull())
test_df.count() / df.count() * 100

0.8961192055071017

In [100]:
df = df.where(df.indrel_1mes.isNotNull()).where(df.tiprel_1mes.isNotNull())

The same applied for `cod_prov` and `nomprov`

In [101]:
test_df = df.where(df.cod_prov.isNull())
test_df = test_df.where(test_df.nomprov.isNull())
test_df.count() / df.count() * 100

0.4878548242194938

In [102]:
df = df.where(df.cod_prov.isNotNull()).where(df.nomprov.isNotNull())

In [134]:
df = df.drop(df.nomprov)

Based on visualization, we can see that `canal_entrada` contains 3 main groups and many other small groups. We'll combine small groups and null values to create another label `Other`

In [110]:
total_count = df.count()

df.groupBy("canal_entrada") \
    .agg(F.count("*").alias("count"), (F.count("*") / total_count * 100).alias("percentage")) \
    .sort(F.desc("count")) \
    .show()

+-------------+-------+-------------------+
|canal_entrada|  count|         percentage|
+-------------+-------+-------------------+
|          KHE|4050971| 30.159980821360953|
|          KAT|3242351| 24.139704771058717|
|          KFC|3089476| 23.001531461976636|
|          KHQ| 591036|  4.400336221793218|
|          KFA| 399299|  2.972830509521941|
|          KHK| 241041| 1.7945800987372325|
|          KHM| 183829| 1.3686296728389225|
|          KHN| 116591| 0.8680344351868466|
|          KHD| 116000|  0.863634366989512|
|          KAS|  85364| 0.6355455526180406|
|          RED|  74953| 0.5580343681807318|
|          KAG|  73516| 0.5473357252034566|
|          KAY|  66959|0.49851804809698913|
|          KAA|  65229|   0.48563798383068|
|          KAB|  62064| 0.4620741668348024|
|          KAE|  50250| 0.3741174736312326|
|          KCC|  48017|0.35749251206668453|
|          KBZ|  45555| 0.3391626171397174|
|          KHL|  45099|0.33576764066258624|
|          KFD|  43691|0.3252849

In [129]:
df = df.fillna("Others", subset=["canal_entrada"])

In [130]:
channels_stats = df.groupBy("canal_entrada") \
    .agg(F.count("*").alias("count"), (F.count("*") / total_count * 100).alias("percentage")) \
    .sort(F.desc("count")) \
    .collect()

In [131]:
channels_to_group = [c.canal_entrada for c in channels_stats if c.percentage < 20]
channels_to_replace = ["Others" for _ in range(len(channels_to_group))]

In [132]:
df = df.replace(channels_to_group, channels_to_replace, subset=["canal_entrada"])

In [133]:
total_count = df.count()

df.groupBy("canal_entrada") \
    .agg(F.count("*").alias("count"), (F.count("*") / total_count * 100).alias("percentage")) \
    .sort(F.desc("count")) \
    .show()

+-------------+-------+------------------+
|canal_entrada|  count|        percentage|
+-------------+-------+------------------+
|          KHE|4050971|30.159980821360953|
|          KAT|3242351|24.139704771058717|
|          KFC|3089476|23.001531461976636|
|       Others|3048812| 22.69878294560369|
+-------------+-------+------------------+



Now, let's look at `segmento`

In [140]:
df.groupBy("segmento") \
    .agg(F.count("*").alias("count"), (F.count("*") / total_count * 100).alias("percentage")) \
    .sort(F.desc("count")) \
    .show()

+------------------+-------+-------------------+
|          segmento|  count|         percentage|
+------------------+-------+-------------------+
| 02 - PARTICULARES|7902964|  58.83854578862847|
|03 - UNIVERSITARIO|4930818| 36.710550708366306|
|          01 - TOP| 558401|  4.157364604838884|
|              NULL|  39427|0.29353889816634043|
+------------------+-------+-------------------+



For this field, we can actually imputate it with the mode value, which is 02 - PARTICUALRES

In [141]:
df = df.fillna("02 - PARTICULARES", subset=["segmento"])

For `renta`, we can imputate it with median value

In [149]:
median_renta = df.select(F.median("renta").alias("median")).collect()[0]

In [151]:
df = df.fillna(median_renta.median, subset="renta")

For those null values in target variables, it is better just to remove them.

In [153]:
df = df.dropna(subset=["ind_nomina_ult1", "ind_nom_pens_ult1"])

Now there are no null values left

In [158]:
df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

+----------+--------+------------+---------------+----+---+----------+---------+----------+------+-----------+-----------+-------+------+-------------+-------+-------+--------+---------------------+-----+--------+-----------------+-----------------+----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+---------------+-----------------+---------------+
|fecha_dato|ncodpers|ind_empleado|pais_residencia|sexo|age|fecha_alta|ind_nuevo|antiguedad|indrel|indrel_1mes|tiprel_1mes|indresi|indext|canal_entrada|indfall|tipodom|cod_prov|ind_actividad_cliente|renta|segmento|ind_ahor_fin_ult1|ind_aval_fin_ult1|ind_cco_fin_ult1|ind_cder_fin_ult1|ind_cno_fin_ult1|ind_ctju_fin_ult1|ind_ctma_fin_ult1|ind_ctop_fin_ult1|ind_ctp

Let's check the schema again.

In [155]:
df.printSchema()

root
 |-- fecha_dato: date (nullable = true)
 |-- ncodpers: integer (nullable = true)
 |-- ind_empleado: string (nullable = true)
 |-- pais_residencia: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- fecha_alta: date (nullable = true)
 |-- ind_nuevo: integer (nullable = true)
 |-- antiguedad: integer (nullable = true)
 |-- indrel: integer (nullable = true)
 |-- indrel_1mes: integer (nullable = true)
 |-- tiprel_1mes: string (nullable = true)
 |-- indresi: integer (nullable = true)
 |-- indext: integer (nullable = true)
 |-- canal_entrada: string (nullable = false)
 |-- indfall: integer (nullable = true)
 |-- tipodom: string (nullable = true)
 |-- cod_prov: string (nullable = true)
 |-- ind_actividad_cliente: integer (nullable = true)
 |-- renta: double (nullable = false)
 |-- segmento: string (nullable = false)
 |-- ind_ahor_fin_ult1: integer (nullable = true)
 |-- ind_aval_fin_ult1: integer (nullable = true)
 |-- ind_cco_fin_ult1: 

The `cod_prov` should be integer.

In [ ]:
df.select("cod_prov").distinct().show()

+--------+
|cod_prov|
+--------+
|      51|
|      15|
|      11|
|      29|
|      42|
|       6|
|      30|
|      34|
|       3|
|       4|
|      22|
|      28|
|       7|
|      16|
|      35|
|      52|
|      47|
|       9|
|      43|
|      31|
+--------+
only showing top 20 rows



In [157]:
df = df.withColumn("cod_prov", df.cod_prov.cast("int"))

Column `tipodom` contains only one value so we can drop this column

In [159]:
df.select("tipodom").distinct().show()

+-------+
|tipodom|
+-------+
|      1|
+-------+



In [160]:
df = df.drop("tipodom")

In [161]:
df.printSchema()

root
 |-- fecha_dato: date (nullable = true)
 |-- ncodpers: integer (nullable = true)
 |-- ind_empleado: string (nullable = true)
 |-- pais_residencia: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- fecha_alta: date (nullable = true)
 |-- ind_nuevo: integer (nullable = true)
 |-- antiguedad: integer (nullable = true)
 |-- indrel: integer (nullable = true)
 |-- indrel_1mes: integer (nullable = true)
 |-- tiprel_1mes: string (nullable = true)
 |-- indresi: integer (nullable = true)
 |-- indext: integer (nullable = true)
 |-- canal_entrada: string (nullable = false)
 |-- indfall: integer (nullable = true)
 |-- cod_prov: integer (nullable = true)
 |-- ind_actividad_cliente: integer (nullable = true)
 |-- renta: double (nullable = false)
 |-- segmento: string (nullable = false)
 |-- ind_ahor_fin_ult1: integer (nullable = true)
 |-- ind_aval_fin_ult1: integer (nullable = true)
 |-- ind_cco_fin_ult1: integer (nullable = true)
 |-- ind_cde

It also seems that these two columns has relationship, and one can be dropped.

In [166]:
df.select("fecha_alta", "antiguedad").distinct().show()

+----------+----------+
|fecha_alta|antiguedad|
+----------+----------+
|2012-08-21|        34|
|2012-05-22|        13|
|2012-07-31|        31|
|2013-04-30|        27|
|2013-10-23|        21|
|2012-10-26|        29|
|2011-08-10|        44|
|2011-08-05|        39|
|2011-09-12|        33|
|2010-10-18|        57|
|2010-12-21|        27|
|2010-07-07|        60|
|2010-06-28|        27|
|2014-06-24|        13|
|2011-10-21|        44|
|2014-09-11|        13|
|2014-04-15|        12|
|2014-06-05|        16|
|2014-12-22|         6|
|2014-11-25|         7|
+----------+----------+
only showing top 20 rows



In [167]:
df = df.drop("fecha_alta")

In [168]:
df.coalesce(1).write.option("header", "true").csv("file:///home/m1nhd3n/Works/DataEngineer/product_recommendations/data/preprocess/handled_missing_new")